In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.5 MB/s eta 0:00:00


In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.4 MB/s eta 0:00:00


In [ ]:
pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=29140fb922e733d8b9e61380b295814888daa134c381e49e34b62bf1ca8ab069
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
pip install transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.0 MB/s eta 0:00:00


In [ ]:
# Import load_dataset
from datasets import load_dataset

In [ ]:
#Load dataset
dataset = load_dataset('json', data_files='FinalNER150.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-bd714f99c97dfa7d/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Import necessary libraries and modules
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import numpy as np
#Check if CUDA is available
import torch
print(torch.cuda.is_available())

In [ ]:
# Define label list and encoding dictionary for NER task
label_list = [
    'O',      # Outside of a named entity
    'B-DATE',
    'I-DATE',
    'B-PER',   # Beginning of a person's name right after another person's name
    'I-PER',   # Person's name
    'B-ORG',   # Beginning of an organisation right after another organisation
    'I-ORG',   # Organisation
    'B-LOC',   # Beginning of a location right after another location
    'I-LOC'    # Location
]
label_encoding_dict = {'O': 0, 'B-DATE':1, 'I-DATE':2, 'B-PER': 3, 'I-PER': 4, 'B-ORG': 5, 'I-ORG': 6, 'B-LOC': 7, 'I-LOC': 8}

# Specify the task, model checkpoint, and batch size
task = "ner"
model_checkpoint = "Davlan/bert-base-multilingual-cased-ner-hrl"
batch_size = 6

#Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Define a function to tokenize and align labels
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. So set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            # Set the label for the first token of each word
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            # For the other tokens in a word, set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# Tokenize and align labels for the training dataset
trained_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [ ]:
import torch
from torch.utils.data import random_split

train_size = int(len(trained_dataset['train']) * 0.8)
val_size = int(len(trained_dataset['train']) * 0.1)
test_size = len(trained_dataset['train']) - train_size - val_size

# Split the training dataset into train, validation, and test datasets
train_dataset, val_dataset, test_dataset = random_split(trained_dataset['train'], [train_size, val_size, test_size], generator=torch.Generator().manual_seed(42))

In [ ]:
#Print the lengths of the datasets
print("train dataset size:", len(train_dataset))
print("train dataset size:", len(val_dataset))
print("train dataset size:", len(test_dataset))

train dataset size: 120
train dataset size: 15
train dataset size: 15


In [ ]:
# Load the pre-trained NER model
model_name = "Davlan/bert-base-multilingual-cased-ner-hrl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

In [ ]:
metric = load_metric("seqeval")

# Define the compute_metrics function to compute evaluation metrics using the seqeval library

def compute_metrics(p):
  # Convert predicted and true labels to their corresponding tags
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    # Compute evaluation metrics
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

<ipython-input-15-2f10eddfcff5>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

# Set up the training arguments and trainer for training the model
args = TrainingArguments(
    "test-ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10  ,
    weight_decay=0.00001,
    logging_steps=1,
)
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
#train the model
trainer.train()
#Evaluate on the test set
test_result = trainer.evaluate(test_dataset)

#Print the evaluation metrics
print(test_result)

# Save the trained model
model_path = "saved_final_model_ner"
model.save_pretrained(model_path)

from google.colab import files
import shutil

# Zip the saved model directory
shutil.make_archive('saved_final_model_ner', 'zip', 'saved_final_model_ner')

# Download the saved model zip file
files.download('saved_final_model_ner.zip')


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.167400,0.132727,0.608491,0.712707,0.656489,0.957055
2,0.072600,0.120610,0.630841,0.745856,0.683544,0.959441
3,0.042700,0.133311,0.781250,0.690608,0.733138,0.970007
4,0.016300,0.143992,0.761006,0.668508,0.711765,0.965917
5,0.004300,0.135446,0.703125,0.745856,0.723861,0.963190
6,0.006300,0.155187,0.763006,0.729282,0.745763,0.965917
7,0.010200,0.172550,0.714286,0.773481,0.742706,0.964554
8,0.006600,0.181815,0.752809,0.740331,0.746518,0.964894
9,0.000400,0.185706,0.748603,0.740331,0.744444,0.964554
10,0.000400,0.183548,0.762712,0.745856,0.754190,0.965576


(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)
(15, 231)


(15, 247)
(15, 247)
{'eval_loss': 0.15847247838974, 'eval_precision': 0.7681818181818182, 'eval_recall': 0.771689497716895, 'eval_f1': 0.7699316628701595, 'eval_accuracy': 0.9661016949152542, 'eval_runtime': 0.4198, 'eval_samples_per_second': 35.732, 'eval_steps_per_second': 7.146, 'epoch': 10.0}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
### Do the same but on the base pre-trained model without fine-tuning to get the metrics of the test set without fine-tuning

# Load the pre-trained NER model
model_name = "Davlan/bert-base-multilingual-cased-ner-hrl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    print(predictions.shape)
    print(labels.shape)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
data_collator = DataCollatorForTokenClassification(tokenizer)
args = TrainingArguments(
    "tested-ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1  ,
    weight_decay=0.00001,
    logging_steps=1, # Set logging_steps to 1 to log at every step
)
trainer = Trainer(
    model,
    args,
    train_dataset=None,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# evaluate on the test set
test_result = trainer.evaluate(test_dataset)

# print the evaluation metrics
print(test_result)



You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(15, 247)
(15, 247)
{'eval_loss': 0.4977824091911316, 'eval_precision': 0.5241935483870968, 'eval_recall': 0.2968036529680365, 'eval_f1': 0.37900874635568516, 'eval_accuracy': 0.9284745762711865, 'eval_runtime': 0.3955, 'eval_samples_per_second': 37.931, 'eval_steps_per_second': 37.931}
